<a href="https://colab.research.google.com/github/Nickguild1993/Business_Py_Explorations/blob/main/Styling_Format_Pandas_SalesData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using the sales total dataset, working on styling in pandas - formatting strings and doing conditional formatting

In [1]:
# import dependencies

import pandas as pd
import numpy as np
from scipy import stats
from random import random
from datetime import datetime as dt
import datetime

In [2]:
# get the data via raw url

url = "https://raw.githubusercontent.com/Nickguild1993/Business_Py_Explorations/main/2018_Sales_Total.csv"

df = pd.read_csv("https://raw.githubusercontent.com/Nickguild1993/Business_Py_Explorations/main/2018_Sales_Total.csv")

df.head(3)

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2018-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2018-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2018-01-01 13:24:58


Quick summary of how much customers have purchased from us and their average purchase amount looks like

In [3]:
df.groupby("name")["ext price"].agg(["mean", "sum"]).round(2)[:5]

,mean,sum
name,,
Barton LLC,1334.62,109438.50
"Cronin, Oberbrunner and Spencer",1339.32,89734.55
"Frami, Hills and Schmidt",1438.47,103569.59
"Fritsch, Russel and Anderson",1385.37,112214.71
"Halvorson, Crona and Champlin",1206.97,70004.36


Messy looking - difficult to discern with all the decimals and lack of formatting.  Da fix - style.format

In [4]:
(df.groupby("name")["ext price"]
 .agg(["mean", "sum"])[:5]
 .style.format('${0:,.2f}'))

,mean,sum
name,,
Barton LLC,"$1,334.62","$109,438.50"
"Cronin, Oberbrunner and Spencer","$1,339.32","$89,734.55"
"Frami, Hills and Schmidt","$1,438.47","$103,569.59"
"Fritsch, Russel and Anderson","$1,385.37","$112,214.71"
"Halvorson, Crona and Champlin","$1,206.97","$70,004.36"


if we wanted to round to zero decimal places, we'd change the format to .style.format('${0:,.0f}')

In [5]:
(df.groupby("name")["ext price"]
 .agg(["mean", "sum"])[:5]
 .style.format('${0:,.0f}'))

,mean,sum
name,,
Barton LLC,"$1,335","$109,438"
"Cronin, Oberbrunner and Spencer","$1,339","$89,735"
"Frami, Hills and Schmidt","$1,438","$103,570"
"Fritsch, Russel and Anderson","$1,385","$112,215"
"Halvorson, Crona and Champlin","$1,207","$70,004"


refer to https://mkaz.blog/code/python-string-format-cookbook/ 

#### More advanced - use **GROUPER** to summarize by month and also calculate how much each month is a percentage of the total annual sales

In [6]:
df.dtypes

account number      int64
name               object
sku                object
quantity            int64
unit price        float64
ext price         float64
date               object
dtype: object

In [7]:
df["date"] = pd.to_datetime(df["date"])

In [8]:
monthly_sales = df.groupby([pd.Grouper(key = "date", freq="M")])["ext price"].agg(["sum"]).reset_index()

monthly_sales["pct_of_total"] = monthly_sales["sum"] / df["ext price"].sum()

In [9]:
monthly_sales

,date,sum,pct_of_total
0,2018-01-31,185361.66,0.091818
1,2018-02-28,146211.62,0.072426
2,2018-03-31,203921.38,0.101012
3,2018-04-30,174574.11,0.086475
4,2018-05-31,165418.55,0.081940
5,2018-06-30,174089.33,0.086235
6,2018-07-31,191662.11,0.094939
7,2018-08-31,153778.59,0.076174
8,2018-09-30,168443.17,0.083438
9,2018-10-31,171495.32,0.084950


Using a dictionary to define a unique formatting string for each column!

In [10]:
format_dict = {"sum" : '${0:,.2f}', "date" : '{:%m-%Y}', "pct_of_total" : '{0:.2%}'}

monthly_sales.style.format(format_dict).hide_index()

date,sum,pct_of_total
01-2018,"$185,361.66",9.18%
02-2018,"$146,211.62",7.24%
03-2018,"$203,921.38",10.10%
04-2018,"$174,574.11",8.65%
05-2018,"$165,418.55",8.19%
06-2018,"$174,089.33",8.62%
07-2018,"$191,662.11",9.49%
08-2018,"$153,778.59",7.62%
09-2018,"$168,443.17",8.34%
10-2018,"$171,495.32",8.49%


#### using conditional formatting with dataframes

highlight the highest number in green and the lowest number in Trinidad (#cd4f39)

In [15]:
(monthly_sales
 .style
 .format(format_dict)
 .hide_index()
 .highlight_max(subset=["sum", "pct_of_total"],color = "lightgreen")
 .highlight_min(subset=["sum", "pct_of_total"],color = "#cd4f39"))

date,sum,pct_of_total
01-2018,"$185,361.66",9.18%
02-2018,"$146,211.62",7.24%
03-2018,"$203,921.38",10.10%
04-2018,"$174,574.11",8.65%
05-2018,"$165,418.55",8.19%
06-2018,"$174,089.33",8.62%
07-2018,"$191,662.11",9.49%
08-2018,"$153,778.59",7.62%
09-2018,"$168,443.17",8.34%
10-2018,"$171,495.32",8.49%


Gradient styling

In [16]:
(monthly_sales.style
 .format(format_dict)
 .background_gradient(subset=["sum"], cmap="BuGn"))

,date,sum,pct_of_total
0,01-2018,"$185,361.66",9.18%
1,02-2018,"$146,211.62",7.24%
2,03-2018,"$203,921.38",10.10%
3,04-2018,"$174,574.11",8.65%
4,05-2018,"$165,418.55",8.19%
5,06-2018,"$174,089.33",8.62%
6,07-2018,"$191,662.11",9.49%
7,08-2018,"$153,778.59",7.62%
8,09-2018,"$168,443.17",8.34%
9,10-2018,"$171,495.32",8.49%


https://matplotlib.org/stable/tutorials/colors/colormaps.html - cmap documentation

#### Styling with bars - drawing bar charts within the columns

In [17]:
(monthly_sales
 .style
 .format(format_dict)
 .hide_index()
 .bar(color = "#FFA07A", vmin = 100_000, subset=["sum"], align= "zero")
 .bar(color = "lightgreen", vmin= 0, subset= ["pct_of_total"], align = "zero")
 .set_caption("2018 sales performance"))

date,sum,pct_of_total
01-2018,"$185,361.66",9.18%
02-2018,"$146,211.62",7.24%
03-2018,"$203,921.38",10.10%
04-2018,"$174,574.11",8.65%
05-2018,"$165,418.55",8.19%
06-2018,"$174,089.33",8.62%
07-2018,"$191,662.11",9.49%
08-2018,"$153,778.59",7.62%
09-2018,"$168,443.17",8.34%
10-2018,"$171,495.32",8.49%


### Sparklines!!!!

In [19]:
!pip install sparklines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 225 kB 6.7 MB/s 
  Created wheel for sparklines: filename=sparklines-0.4.2-py3-none-any.whl size=224607 sha256=e282f4f24bc38ab8acdf0b72b5b5422ca92147944793fcc86c143cbcb81479df
  Stored in directory: /root/.cache/pip/wheels/12/b0/82/05c8f8b4f22cc0b9f0acd430e13223ea37557123a0d8b25623
Successfully built sparklines


In [20]:
import sparklines

def sparkline_str(x):
  bins = np.histogram(x)[0]
  s1 = "".join(sparklines(bins))
  return s1
sparkline_str.__name__="sparkline"

Not sure what's going on here?

In [23]:
df.groupby("name")["ext price"].agg(["mean", sparkline_str])

/usr/local/lib/python3.7/dist-packages/pandas/core/groupby/generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


TypeError: ignored